In [1]:
import os
import pickle
from datetime import datetime

import tensorflow as tf

print(datetime.now().strftime("%Y/%m/%d %H:%M:%S"), "Process started", flush=True)

# Initial Setup for GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


2024-08-10 21:53:05.744987: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-10 21:53:05.746220: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-10 21:53:05.770956: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-10 21:53:05.771448: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-10 21:53:06.214651: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

2024/08/10 21:53:06 Process started


2024-08-10 21:53:06.605717: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-10 21:53:06.605735: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: pn01
2024-08-10 21:53:06.605737: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: pn01
2024-08-10 21:53:06.605779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.183.1
2024-08-10 21:53:06.605789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.183.1
2024-08-10 21:53:06.605791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.183.1


In [2]:
Smiles_Path = "train_data/decimal_results.csv"

with open(Smiles_Path, "r") as txt_file:
    smiles = txt_file.read()

In [3]:
for line in smiles.split("\n"):
    print(line)

ChemistryArt_bnw_0,C1=CC2=C(C=C1)C(=O)N([C@@H]3CCC(=O)NC3=O)C2=O
ChemistryArt_bnw_1,C1=CC2=C(C=C1)C(=O)N([C@H]3CCC(=O)NC3=O)C2=O
ChemistryArt_orig_0,C1=CC2=C(C=C1)C(=O)N([C@@H]3CCC(=O)NC3=O)C2=O
ChemistryArt_orig_1,C1=CC2=C(C=C1)C(=O)N([C@H]3CCC(=O)NC3=O)C2=O



In [5]:

# storing the captions and the image name in vectors
all_smiles = []
all_img_name = []
for line in smiles.split("\n"):
    # Split the ID and SMILES to separate tokens
    tokens = line.split(",")
    # Check if tokens have at least 2 elements (ID and SMILES)
    if len(tokens) < 2:
        continue  # Skip lines that don't have both an ID and a SMILES string
    
    image_id = str(tokens[0]) + ".png"
    # Add start and end annotations to SMILES string
    try:
        SMILES_ = (
            "<start> " + str(tokens[1].replace("][", "] [").rstrip()) + " <end>"
        )
    except IndexError as e:
        print(e, flush=True)

    # PATH of training images
    full_image_path = "train_data/" + image_id

    all_img_name.append(full_image_path)
    all_smiles.append(SMILES_)

train_captions, img_name_vector = (all_smiles, all_img_name)

In [9]:
train_captions

['<start> C1=CC2=C(C=C1)C(=O)N([C@@H]3CCC(=O)NC3=O)C2=O <end>',
 '<start> C1=CC2=C(C=C1)C(=O)N([C@H]3CCC(=O)NC3=O)C2=O <end>',
 '<start> C1=CC2=C(C=C1)C(=O)N([C@@H]3CCC(=O)NC3=O)C2=O <end>',
 '<start> C1=CC2=C(C=C1)C(=O)N([C@H]3CCC(=O)NC3=O)C2=O <end>']

In [10]:
img_name_vector

['train_data/ChemistryArt_bnw_0.png',
 'train_data/ChemistryArt_bnw_1.png',
 'train_data/ChemistryArt_orig_0.png',
 'train_data/ChemistryArt_orig_1.png']

In [12]:

print("Total number of selected SMILES Strings: ", len(train_captions), flush=True)

# Defining the maximum number of tokens to generate
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

# choosing the top 500 words from the vocabulary
top_k = 500
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=top_k, oov_token="<unk>", lower=False, filters=""
)

tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

tokenizer.word_index["<pad>"] = 0
tokenizer.index_word[0] = "<pad>"

# padding each vector to the max_length of the captions, if the max_length parameter is not provided, pad_sequences calculates that automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(
    train_seqs, padding="post"
)

# calculating the max_length used to store the attention weights
max_length = calc_max_length(train_seqs)

# img_name_train, img_name_val, cap_train, cap_val = (
#     img_name_vector[0:1340000],
#     img_name_vector[1340000:1440000],
#     cap_vector[0:1340000],
#     cap_vector[1340000:1440000],
# )

total_images = len(train_captions)
split_index = int(0.9 * total_images)  # 90 images for training, 10 for validation
img_name_train = img_name_vector[:split_index]  # 90 images
img_name_val = img_name_vector[split_index:]    # 10 images
cap_train = cap_vector[:split_index]  # Corresponding 90 captions
cap_val = cap_vector[split_index:]    # Corresponding 10 captions



print(
    str(len(img_name_train)),
    str(len(img_name_val)),
    str(len(cap_train)),
    str(len(cap_val)),
)

    # return img_name_train, cap_train, tokenizer, max_length, img_name_val

Total number of selected SMILES Strings:  4
3 1 3 1


In [29]:
start = 0
end = 2
start_x = 0

for i in range(int(len(img_name_vector) / end)):

    imgs_path = img_name_vector[start : (start + end)]
    caps_path = cap_vector[start : (start + end)]


    
    with open(
        "pkl/Images_" + str(start_x) + "_" + str(start_x) + ".pkl", "wb"
    ) as file:
        pickle.dump(imgs_path, file)

    with open(
        "pkl/Capts_" + str(start_x) + "_" + str(start_x) + ".pkl", "wb"
    ) as file:
        pickle.dump(caps_path, file)

    print("Total Train_Images: ", len(imgs_path), flush=True)
    print("Total SELFIES_Images: ", len(caps_path), flush=True)

    start = start + len(imgs_path)
    start_x = start_x + 1

Total Train_Images:  2
Total SELFIES_Images:  2
Total Train_Images:  2
Total SELFIES_Images:  2


In [30]:
ls pkl

Capts_0_0.pkl  Capts_1_1.pkl  Images_0_0.pkl  Images_1_1.pkl


In [19]:
caps_path

array([[2, 4, 3],
       [2, 5, 3]], dtype=int32)

In [1]:
import pickle

In [3]:
max_length = pickle.load(open("pkl/max_length_TPU_Stereo.pkl", "rb"))
max_length

3